<a href="https://colab.research.google.com/github/magdalena-b/Bairon/blob/master/bart_prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np

import re
import math
import random

In [2]:
data_file = "cummings.txt"

In [3]:
with open(data_file, 'r', encoding = 'utf-8') as f:
    lyrics = f.read()

In [4]:
def create_lyric_pairs(lyrics):
  
    source = []
    target = []
    
    lyrics_ = re.sub(r' \([^)]*\)', '', lyrics)

    lyrics_ = lyrics_.split('\n')

    # Clean up extra spaces in the lyrics
    counter = 0
    for i in range(len(lyrics_)-1):
        if lyrics_[counter] == "" and lyrics[counter+1] == "":
            lyrics_[counter+1].pop()
        counter += 1
    
    counter = 0
    for i in range(len(lyrics_)-1):
        if lyrics_[counter] == "" and lyrics[counter+1] == "":
            lyrics_[counter+1].pop()
        counter += 1    
    # replace a "" with the word break
    for i in range(len(lyrics_)):
        if lyrics_[i] == "":
            lyrics_[i] = "break"
    
    counter = 0
    for i in range(len(lyrics_) - 1):
        if lyrics_[counter] != "break" and lyrics_[counter+1] != "break":
            # Use one line and target is the next line
            source.append(lyrics_[counter])
            target.append(lyrics_[counter+1])
            
        counter += 1
            
    return source, target

def noise_sentence(sentence_, percent_words, replacement_token = "<mask>"):
    '''
    Args: sentence - the sentence to noise
          percent_words - the percent of words to remove
    '''
    # Create a list item and copy
    sentence_ = sentence_.split(' ')
    sentence = sentence_.copy()
    
    num_words = math.ceil(len(sentence) * percent_words)
    
    # # Create an array of tokens to sample from; don't include the last word as an option because in the case of lyrics
    # # that word is often a rhyming word and plays an important role in song construction
    # sample_tokens = set(np.arange(0, np.maximum(1, len(sentence)-1)))

    #include the last word
    sample_tokens = set(np.arange(0, np.maximum(1, len(sentence))))
    
    words_to_noise = random.sample(sample_tokens, num_words)
    
    # Remove redundant spaces
    sentence = re.sub(r' {2,5}', ' ', ' '.join(sentence))
    
    # Combine concurrent <mask> tokens into a single token; this just does two rounds of this; more could be done
    sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
    sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
    return sentence

In [5]:
source_lyrics = []
target_lyrics = []

with open(data_file, 'r', encoding = 'utf-8') as f:
    lyrics = f.read()
    a, b = create_lyric_pairs(lyrics)
    source_lyrics.extend(a)
    target_lyrics.extend(b)

In [6]:
lyrics_df = pd.DataFrame({"source":source_lyrics, "target":target_lyrics})

In [7]:
lyrics_df = lyrics_df.drop_duplicates(subset = 'source')
lyrics_df = lyrics_df.drop_duplicates(subset = 'target')

In [8]:
lyrics_df = lyrics_df[~lyrics_df.source.str.contains(":")]
lyrics_df = lyrics_df[~lyrics_df.target.str.contains(":")]
lyrics_df = lyrics_df[lyrics_df.target != "break"]

In [9]:
lyrics_df['source'] = lyrics_df['source'].apply(lambda x: noise_sentence(x, 0.25))

In [10]:
lyrics_df.to_csv("lyrics_simple_noised_cummings_w_last_word.csv", index = False)